<a href="https://colab.research.google.com/github/Chu-Yichen/QM2-Group-19/blob/main/Spatial_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Spatial Analysis**

In [ ]:
def spatial_analysis(df_siteyear, pollutant_name, pollutant_unit, site_locations):

    # Merge with site coordinates
    site_locations_reset = site_locations.reset_index()
    df_siteyear_merged = pd.merge(
        df_siteyear,
        site_locations_reset[['Site Id', 'Latitude', 'Longitude']],
        on='Site Id', how='inner'
    )

    # Clean data and define pollutant value range for color scaling
    df_siteyear_merged.dropna(subset=['annual_pollutant', 'Latitude', 'Longitude'], inplace=True)
    vmin, vmax = df_siteyear_merged['annual_pollutant'].min(), df_siteyear_merged['annual_pollutant'].max()

    # Define custom color map (Green -> Yellow -> Red)
    my_cmap = mcolors.LinearSegmentedColormap.from_list('colormap', ['green', 'yellow', 'red'])

    # Function to plot map for a selected year
    def plot_map_for_year(year):
        """Plot annual pollutant data for the given year."""
        df_year = df_siteyear_merged[df_siteyear_merged['Year'] == year]
        plt.figure(figsize=(8, 8))

        # Create Basemap focused on Atlanta
        m = Basemap(projection='lcc', resolution='l', lat_0=BASEMAP_LAT, lon_0=BASEMAP_LON, width=4e5, height=4e5)
        m.drawcoastlines(), m.drawcountries(), m.drawstates()

        # Mark airport location with a star
        x_airport, y_airport = m(AIRPORT_LON, AIRPORT_LAT)
        m.plot(x_airport, y_airport, marker='*', color='black', markersize=14, label=f'{AIRPORT_CODE} Airport')

        # Add a radius circle
        m.tissot(AIRPORT_LON, AIRPORT_LAT, NEARBY_RADIUS / 111.0, 100, facecolor='none', edgecolor='red', linewidth=2, alpha=0.8)

        # Plot pollutant levels for sites
        x, y = m(df_year['Longitude'].values, df_year['Latitude'].values)
        sc = m.scatter(x, y, c=df_year['annual_pollutant'], cmap=my_cmap, vmin=vmin, vmax=vmax, s=60, alpha=0.8, edgecolors='white', linewidths=0.5)

        # Add colorbar and title
        cbar = plt.colorbar(sc, orientation='vertical', shrink=0.7, pad=0.03)
        cbar.set_label(f'Annual Average {pollutant_name} ({pollutant_unit})')
        plt.title(f"Annual {pollutant_name} in {CITY_NAME} Region - {year}")
        plt.legend(loc='lower left')
        plt.show()

    # Create interactive dropdown for year selection
    all_years = sorted(df_siteyear_merged['Year'].unique())
    interact(
        plot_map_for_year,
        year=widgets.Dropdown(options=all_years, value=all_years[0], description='Select Year:')
    )